# 如何添加线程级持久化（函数式API）

!!!信息“先决条件”

本指南假设您熟悉以下内容：

- [功能 API](../../concepts/function_api/)
- [持久性](../../概念/持久性/)
- [记忆](../../概念/记忆/)
- [聊天模型](https://js.langchain.com/docs/concepts/chat_models/)

许多人工智能应用程序需要内存来在同一线程上的多个交互之间共享上下文（../../concepts/persistence#threads）（例如，多轮对话）。在 LangGraph 函数式 API 中，这种内存可以使用 [线程级持久性](/langgraphjs/concepts/persistence) 添加到任何 [entrypoint()](/langgraphjs/reference/functions/langgraph.entrypoint-1.html) 工作流程中。

创建 LangGraph 工作流程时，您可以使用 [checkpointer](/langgraphjs/reference/classes/checkpoint.BaseCheckpointSaver.html) 将其设置为保留其结果：


1. 创建检查点实例：
    ```ts
    import { MemorySaver } from "@langchain/langgraph";
    
    const checkpointer = new MemorySaver();
    ```
2. 将 `checkpointer` 实例传递给 `entrypoint()` 包装函数：
    ```ts
    import { entrypoint } from "@langchain/langgraph";
    const workflow = entrypoint({
      name: "workflow",
      checkpointer,
    }, async (inputs) => {
      ...
    });
    ```
3. 从工作流中之前的执行中检索 `previous` 状态：
    ```ts
    import { entrypoint, getPreviousState } from "@langchain/langgraph";
    
    const workflow = entrypoint({
      name: "workflow",
      checkpointer,
    }, async (inputs) => {
      const previous = getPreviousState();
      const result = doSomething(previous, inputs);
      ...
    });
    ```
4. （可选）选择将从工作流程返回哪些值以及检查点将哪些值保存为 `previous`：
    ```ts
    import { entrypoint, getPreviousState } from "@langchain/langgraph";
    
    const workflow = entrypoint({
      name: "workflow",
      checkpointer,
    }, async (inputs) => {
      const previous = getPreviousState();
      const result = doSomething(previous, inputs);
      ...
      return entrypoint.final({
        value: result,
        save: combineState(inputs, result),
      });
    });
    ```
本指南展示了如何将线程级持久性添加到工作流程中。

!!!提示“注意”

如果您需要在多个对话或用户之间 __shared__ 的内存（跨线程持久性），请查看此[操作指南](../cross-thread-persistence-function)。

!!!提示“注意”

如果您需要向 `StateGraph` 添加线程级持久性，请查看此[操作指南](../persistence)。

## 设置

!!!注意兼容性

本指南需要 `@langchain/langgraph>=0.2.42`。

首先，安装本示例所需的依赖项：
```bash
npm install @langchain/langgraph @langchain/anthropic @langchain/core zod
```
接下来，我们需要为 Anthropic 设置 API 密钥（我们将使用的 LLM）：
```typescript
process.env.ANTHROPIC_API_KEY = "YOUR_API_KEY";
```
!!!提示“为 LangGraph 开发设置 [LangSmith](https://smith.langchain.com)”

注册 LangSmith 以快速发现问题并提高 LangGraph 项目的性能。LangSmith 允许您使用跟踪数据来调试、测试和监控使用 LangGraph 构建的 LLM 应用程序 — 在[此处](https://docs.smith.langchain.com)了解有关如何开始的更多信息

## 示例：具有短期记忆的简单聊天机器人

我们将使用一个包含单个任务的工作流程来调用[聊天模型](https://js.langchain.com/docs/concepts/chat_models/)。

让我们首先定义我们将使用的模型：

In [1]:
import { ChatAnthropic } from "@langchain/anthropic";

const model = new ChatAnthropic({
  model: "claude-3-5-sonnet-latest",
});

现在我们可以定义我们的任务和工作流程。为了添加持久性，我们需要将 [Checkpointer](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.base.BaseCheckpointSaver) 传递给 [entrypoint()](/langgraphjs/reference/functions/langgraph.entrypoint-1.html) 包装函数。

In [2]:
import type { BaseMessage, BaseMessageLike } from "@langchain/core/messages";
import {
  addMessages,
  entrypoint,
  task,
  getPreviousState,
  MemorySaver,
} from "@langchain/langgraph";

const callModel = task("callModel", async (messages: BaseMessageLike[]) => {
  const response = model.invoke(messages);
  return response;
});

const checkpointer = new MemorySaver();

const workflow = entrypoint({
  name: "workflow",
  checkpointer,
}, async (inputs: BaseMessageLike[]) => {
  const previous = getPreviousState<BaseMessage>() ?? [];
  const messages = addMessages(previous, inputs);
  const response = await callModel(messages);
  return entrypoint.final({
    value: response,
    save: addMessages(messages, response),
  });
});

如果我们尝试使用此工作流程，对话的上下文将在交互过程中保持不变。

!!!注释 注释

如果您使用 LangGraph Cloud 或 LangGraph Studio，则不需要将检查指针传递给 `entrypoint` 包装器，因为它是自动完成的。

这在实践中是如何运作的：

In [3]:
const config = {
  configurable: { thread_id: "1" },
  streamMode: "values" as const,
};
const inputMessage = { role: "user", content: "hi! I'm bob" };

const stream = await workflow.stream(
  [inputMessage],
  config,
);

for await (const chunk of stream) {
  console.log("=".repeat(30), `${chunk.getType()} message`, "=".repeat(30));
  console.log(chunk.content);
}

============================== ai message ==============================
Hi Bob! I'm Claude. Nice to meet you! How can I help you today?


您始终可以恢复以前的线程：

In [4]:
const followupStream = await workflow.stream(
  [{ role: "user", content: "what's my name?" }], 
  config,
);

for await (const chunk of followupStream) {
  console.log("=".repeat(30), `${chunk.getType()} message`, "=".repeat(30));
  console.log(chunk.content);
}

============================== ai message ==============================
Your name is Bob - you just told me that in your first message.


如果我们想开始一个新的对话，我们可以传入不同的`thread_id`。噗！所有的记忆都消失了！

In [5]:
const newStream = await workflow.stream(
  [{ role: "user", content: "what's my name?" }],
  {
    configurable: {
      thread_id: "2",
    },
    streamMode: "values",
  },
);

for await (const chunk of newStream) {
  console.log("=".repeat(30), `${chunk.getType()} message`, "=".repeat(30));
  console.log(chunk.content);
}

============================== ai message ==============================
I don't know your name as we just started chatting. Would you like to introduce yourself?


!!!提示“流令牌”

如果您想从聊天机器人流式传输 LLM 令牌，您可以使用 `streamMode: "messages"`。查看此[操作指南](../stream-tokens) 以了解更多信息。